In [1]:
# load SQL libraries and other necessities
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display
import sqlalchemy as sa
import plotly.plotly as py # interactive graphing
from plotly.graph_objs import Bar, Scatter, Marker, Layout 
%load_ext sql
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# create SQL database 
con = sa.create_engine('sqlite:///greentaxi4.db')
chunksize=10000
chunks = pd.read_csv('green_tripdata_2013-09.csv', chunksize=chunksize)

# in the next box, we connect to the database

In [3]:
%%sql 
sqlite:///greentaxi4.db

u'Connected: None@greentaxi4.db'

In [4]:
# load data into database
j=0
for chunk in chunks:
    chunk.to_sql('grt5', if_exists='append', con=con)
    j+=1
    print 'completed {} rows'.format(j*10000)

completed 10000 rows
completed 20000 rows
completed 30000 rows
completed 40000 rows
completed 50000 rows


In [5]:
%%sql
SELECT * FROM grt5 LIMIT '3'

Done.


level_0,level_1,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,Total_amount,Payment_type,Trip_type
2,2013-09-01 00:02:00,2013-09-01 00:54:51,N,1,-73.9524078369,40.8107261658,-73.9839401245,40.67628479,5,14.35,50.5,0.5,0.5,10.3,0.0,None,61.8,1,None,None,None
2,2013-09-01 00:02:34,2013-09-01 00:20:59,N,1,-73.9630203247,40.7118339539,-73.9666442871,40.6816902161,1,3.24,15.0,0.5,0.5,0.0,0.0,None,16.0,2,None,None,None
2,2013-09-01 00:03:06,2013-09-01 00:28:03,N,1,-73.843460083,40.7559509277,-73.9892120361,40.7405281067,1,11.27,34.0,0.5,0.5,8.07,5.33,None,48.4,1,None,None,None


In [6]:
# next we delete the last 2 columns from the table

In [7]:
%%sql
CREATE TEMPORARY TABLE IF NOT EXISTS grt5_backup(level_0,level_1,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,
                                  RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,
                                  Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,
                                  Ehail_fee,Total_amount);
INSERT INTO grt5_backup SELECT level_0,level_1,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,
                                  RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,
                                  Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,
                                  Ehail_fee,Total_amount FROM grt5;
DROP TABLE grt5;
CREATE TABLE IF NOT EXISTS grt5(level_0,level_1,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,
                                  RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,
                                  Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,
                                  Ehail_fee,Total_amount);
INSERT INTO grt5 SELECT level_0,level_1,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,
                                  RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,
                                  Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,
                                  Ehail_fee,Total_amount FROM grt5_backup;
DROP TABLE grt5_backup;


Done.
49647 rows affected.
Done.
Done.
49647 rows affected.
Done.


[]

In [8]:
# next, we rename the columns

In [9]:
%%sql

ALTER TABLE grt5 RENAME TO grt5_backup1;
CREATE TABLE grt5(VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,
                                  RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,
                                  Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,
                                  Ehail_fee,Total_amount,Payment_type,Trip_type);
INSERT INTO grt5(VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,
                                  RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,
                                  Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,
                                  Ehail_fee,Total_amount,Payment_type,Trip_type) 
SELECT level_0,level_1,VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,
                                  RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,
                                  Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,
                                  Ehail_fee,Total_amount FROM grt5_backup1;
DROP TABLE grt5_backup1;


Done.
Done.
49647 rows affected.
Done.


[]

In [10]:
# change to 'Datetime' type

In [11]:
%%sql
ALTER TABLE grt5 RENAME TO grt5_backup2;
CREATE TABLE IF NOT EXISTS grt5(VendorID,lpep_pickup_datetime DATETIME,Lpep_dropoff_datetime DATETIME,Store_and_fwd_flag,
                                  RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,
                                  Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,
                                  Ehail_fee,Total_amount,Payment_type,Trip_type);
INSERT INTO grt5(VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,
                                  RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,
                                  Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,
                                  Ehail_fee,Total_amount,Payment_type,Trip_type) 
SELECT VendorID,lpep_pickup_datetime DATETIME,Lpep_dropoff_datetime DATETIME,Store_and_fwd_flag,
                                  RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,
                                  Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,
                                  Ehail_fee,Total_amount,Payment_type,Trip_type FROM grt5_backup2;
DROP TABLE grt5_backup2;

Done.
Done.
49647 rows affected.
Done.


[]

In [12]:
# To merge the weather table with the taxi table, I'll do a left join using the dates from the 2 tables.
# However, the datetimes from both tables must be in the same format to facilitate the left join. Thus, I do some wrangling
# the datetime for the taxi data to put it into a format consistent with the weather data datetimes

In [13]:
# add new column to table in preparation for cleaned datetimes

In [14]:
%%sql
ALTER TABLE grt5 ADD COLUMN date_cleaned DATETIME;


Done.


[]

In [15]:
%%sql
UPDATE grt5 
SET date_cleaned =  substr(Lpep_dropoff_datetime, 1, 4)||""||substr(Lpep_dropoff_datetime, 5,2)||""||substr(Lpep_dropoff_datetime, 7,2)||""||
substr(Lpep_dropoff_datetime,9,2)||""||substr(Lpep_dropoff_datetime,11,3)


49647 rows affected.


[]

In [16]:
# LOAD WEATHER DATA and add its table to greentaxi database

In [17]:
chunksize=100
chunks2 = pd.read_csv('weather_central_park_09_2013.csv', chunksize=chunksize)

In [18]:
j=0
for chunk2 in chunks2:
    chunk2.to_sql('weather_gr_Sep', if_exists='append', con=con)
    j+=1
    print 'completed {} rows'.format(j*chunksize)

completed 100 rows
completed 200 rows
completed 300 rows
completed 400 rows
completed 500 rows
completed 600 rows
completed 700 rows
completed 800 rows
completed 900 rows


In [19]:
# keep only the datetime and temp columns from weather table
# NB: use backticks when column name has symbols/hyphens ,etc

In [20]:
%%sql
CREATE TABLE IF NOT EXISTS weather_gr_Sep_clean2(`YR--MODAHRMN`,`TEMP`);
INSERT INTO weather_gr_Sep_clean2 SELECT `YR--MODAHRMN`,`TEMP` FROM weather_gr_Sep;
SELECT * FROM weather_gr_Sep_clean2 LIMIT '3'

Done.
849 rows affected.
Done.


YR--MODAHRMN,TEMP
2.01309010051e+11,82
2.01309010151e+11,81
2.01309010251e+11,80


In [21]:
# rename column to 'date' and convert to datetime format.

In [22]:
%%sql
CREATE TABLE weather_c2(Date DATETIME,Temp);
INSERT INTO weather_c2(Date,Temp) 
SELECT `YR--MODAHRMN`,`TEMP` FROM weather_gr_Sep_clean2;

Done.
849 rows affected.


[]

In [23]:
# add new column to table in preparation for cleaned datetimes

In [24]:
%%sql
ALTER TABLE weather_c2 ADD COLUMN date_cleaned DATETIME;

Done.


[]

In [25]:
# Fill the new column with cleaned datetimes

In [26]:
%%sql
UPDATE weather_c2 
SET date_cleaned =  substr(Date, 1, 4)||"-"||substr(Date, 5,2)||"-"||substr(Date, 7,2)||" "||
substr(Date,9,2)

849 rows affected.


[]

In [27]:
# MERGE weather and taxi tables

In [28]:
%%sql
CREATE TABLE `mergetable` (VendorID,
                        lpep_pickup_datetime,
                        Lpep_dropoff_datetime,
                        Store_and_fwd_flag,
                        RateCodeID,
                        Pickup_longitude,
                        Pickup_latitude,
                        Dropoff_longitude,
                        Dropoff_latitude,
                        Passenger_count,
                        Trip_distance,
                        Fare_amount,
                        Extra,
                        MTA_tax,
                        Tip_amount,
                        Tolls_amount,
                        Ehail_fee,
                        Total_amount,
                        Payment_type,
                        Trip_type,
                        date_cleaned,
                        Date,
                        Temp,
                        date_cleaned_1);

Done.


[]

In [29]:
%%sql
INSERT INTO `mergetable` 
SELECT * FROM grt5 LEFT JOIN weather_c2
ON grt5.date_cleaned = weather_c2.date_cleaned;

55728 rows affected.


[]

In [30]:
# create a column 'dayofweek'

In [31]:
%%sql
CREATE TABLE `mergetable2` (VendorID,
                        lpep_pickup_datetime,
                        Lpep_dropoff_datetime,
                        Store_and_fwd_flag,
                        RateCodeID,
                        Pickup_longitude,
                        Pickup_latitude,
                        Dropoff_longitude,
                        Dropoff_latitude,
                        Passenger_count,
                        Trip_distance,
                        Fare_amount,
                        Extra,
                        MTA_tax,
                        Tip_amount,
                        Tolls_amount,
                        Ehail_fee,
                        Total_amount,
                        Payment_type,
                        Trip_type,
                        date_cleaned,
                        Date,
                        Temp,
                        date_cleaned_1,
                         dayofweek );


Done.


[]

In [32]:
%%sql
INSERT INTO `mergetable2` 
select *, 
  case cast (strftime('%w', lpep_pickup_datetime) as integer)
  when 0 then 'Sunday'
  when 1 then 'Monday'
  when 2 then 'Tuesday'
  when 3 then 'Wednesday'
  when 4 then 'Thursday'
  when 5 then 'Friday'
  else 'Saturday' end as dayofweek
from mergetable;


55728 rows affected.


[]

In [33]:
# Get 'hour' column for each trip

In [34]:
%%sql
CREATE TABLE `mergetable3` (VendorID,
                        lpep_pickup_datetime,
                        Lpep_dropoff_datetime,
                        Store_and_fwd_flag,
                        RateCodeID,
                        Pickup_longitude,
                        Pickup_latitude,
                        Dropoff_longitude,
                        Dropoff_latitude,
                        Passenger_count,
                        Trip_distance,
                        Fare_amount,
                        Extra,
                        MTA_tax,
                        Tip_amount,
                        Tolls_amount,
                        Ehail_fee,
                        Total_amount,
                        Payment_type,
                        Trip_type,
                        date_cleaned,
                        Date,
                        Temp,
                        date_cleaned_1,
                         dayofweek,
                           hour);


Done.


[]

In [35]:
%%sql
INSERT INTO `mergetable3` 
select *,strftime('%H', `lpep_pickup_datetime`) 
as hour 
from mergetable2;

55728 rows affected.


[]

In [36]:
%%sql
SELECT * FROM mergetable3 LIMIT '3'

Done.


VendorID,lpep_pickup_datetime,Lpep_dropoff_datetime,Store_and_fwd_flag,RateCodeID,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,Trip_distance,Fare_amount,Extra,MTA_tax,Tip_amount,Tolls_amount,Ehail_fee,Total_amount,Payment_type,Trip_type,date_cleaned,Date,Temp,date_cleaned_1,dayofweek,hour
2,2013-09-01 00:02:00,2013-09-01 00:54:51,N,1,-73.9524078369,40.8107261658,-73.9839401245,40.67628479,5,14.35,50.5,0.5,0.5,10.3,0.0,None,61.8,1,None,2013-09-01 00,201309010051,82,2013-09-01 00,Sunday,00
2,2013-09-01 00:02:34,2013-09-01 00:20:59,N,1,-73.9630203247,40.7118339539,-73.9666442871,40.6816902161,1,3.24,15.0,0.5,0.5,0.0,0.0,None,16.0,2,None,2013-09-01 00,201309010051,82,2013-09-01 00,Sunday,00
2,2013-09-01 00:03:06,2013-09-01 00:28:03,N,1,-73.843460083,40.7559509277,-73.9892120361,40.7405281067,1,11.27,34.0,0.5,0.5,8.07,5.33,None,48.4,1,None,2013-09-01 00,201309010051,82,2013-09-01 00,Sunday,00
